<a href="https://colab.research.google.com/github/mariamstoica/no10hackathon/blob/main/No10Team10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# put all import statements here!
import os
import numpy as np
import requests
from pprint import pprint
import ipywidgets as widgets
from IPython.display import display

from google.colab import userdata
!pip install newscatcherapi-python-sdk==6.0.2 --quiet
!pip install openai --quiet
from newscatcherapi_client import Newscatcher, ApiException
from polling import get_relevant_polling
from summary import get_summary

from openai import OpenAI
import warnings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [4]:
# API infos
openai_key = userdata.get('OPENAI_API_KEY')
newscatcher_key = userdata.get('NEWSCATCHER_API_KEY')
anthropic_key = userdata.get('ANTHROPIC_API_KEY')



# Polling Data

In [5]:
def get_polling(demographic, region, keword):
  polls = get_relevant_polling(demographic, region, keyword.lower())
  for question in polls:
    print(question)
    print('Categories: {}, {}'.format(demographic, region))
    for cat in polls[question][demographic]:
        print(cat + ': ' + str(polls[question][demographic][cat]) + ', ' + str(polls[question][region][cat]))
    print('\n')

## Summary

In [6]:
def get_overall_summary(keyword, region):
  summary = get_summary(keyword, region)
  print(summary)

def get_questions_all(keyword, region):
  questions = get_questions(keyword, region)
  print(questions)

## Get Questions

In [7]:
# from questions import get_questions

In [8]:
# questions = get_questions(keyword, region)

NameError: name 'keyword' is not defined

In [ ]:
# print(questions)

In [ ]:
# !pip install openai

In [ ]:
# generate mp3 of a question



warnings.filterwarnings('ignore', category=DeprecationWarning)

client = OpenAI(api_key=openai_key)

speech_file_path = "speech.mp3"
response = client.audio.speech.create(
  model="tts-1",
  voice="fable",
  input=questions.split('\n')[0][2:]
)

response.stream_to_file(speech_file_path)

# Focus Group Data


# News Articles


In [9]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def get_article_summary(article_description):
  url = "https://api.openai.com/v1/chat/completions"
  text = 'Provide a two sentence summary of the following text: ' + article_description
  headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai_key}"
    }
  data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": text
            }
        ]
    }
  response = requests.post(url, headers=headers, json=data)
  return response.json()['choices'][0]['message']['content']

def get_article_sentiment(article_description):
  url = "https://api.openai.com/v1/chat/completions"
  headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai_key}"}
  data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {
                "role": "system",
                "content": "You will be provided with a text, and your task is to classify its sentiment as positive, neutral, or negative."
            },
            {
                "role": "user",
                "content": article_description
            }
        ]
    }
  response = requests.post(url, headers=headers, json=data)
  sentiment = response.json()['choices'][0]['message']['content']

  if 'positive' in sentiment.lower():
      sentiment = 'Positive'
  elif 'negative' in sentiment.lower():
      sentiment = 'Negative'
  elif 'neutral' in sentiment.lower():
      sentiment = 'Neutral'
  else:
      sentiment = 'None'

  return sentiment

In [10]:


def get_newscatcher_outputs(keyword, region):
  url = 'https://v3-api.newscatcherapi.com/api/search?'
  if region == 'North-East':
    all_regions = ['North-East', 'Yorkshire', 'Lincolnshire', 'Newcastle']
  else:
    all_regions = [region]

  def create_or_list(word_list):
    return_str = word_list[0]
    if len(word_list) > 1:
      for word in word_list[1:]:
        return_str += ' OR ' + word
      return return_str

  query = keyword + ' (' + create_or_list(all_regions) + ')'
  # print("%s: %s" % (k, v))
  params = {'q': query, 'lang' : ['en'], 'countries' : ["GB"], 'from_' : '10 days ago',
            # Tuesday April 16th 4pm
            'sort_by': 'relevancy',
            'clustering_enabled': True, 'clustering_threshold': 0.7,
            'clustering_variable': 'content'}
  headers = {'x-api-token': newscatcher_key}
  response = requests.get(url, params=params, headers=headers)
  json_data = response.json() if response and response.status_code == 200 else None

  outputs = []
  if json_data is not None:
    clusters = json_data['clusters'][0:6]

    for cluster in clusters:
      cluster_dict = dict()
      first_article = cluster['articles'][0]
      article_description = first_article['description']
      summary = get_article_summary(article_description)
      sentiment = get_article_sentiment(article_description)
      title = first_article['title']
      cluster_dict['title'] = color.BOLD + title + color.END
      cluster_dict['summary'] = summary
      if sentiment == 'Positive':
        cluster_dict['sentiment'] = color.GREEN + sentiment + color.END
      elif sentiment == 'Negative':
        cluster_dict['sentiment'] = color.RED + sentiment + color.END
      else:
        cluster_dict['sentiment'] = color.BLUE + sentiment + color.END
      cluster_dict['url'] = first_article['link']
      outputs.append(cluster_dict)
      # print("%s: %s \n %s" % (sentiment, title, summary))
      # pprint([sentiment, title, summary])

  for output in outputs:
    print('Title: ' + output['title'])
    print('Sentiment: ' + output['sentiment'])
    print('Summary: ' + output['summary'])
    print('URL: ' + output['url'])
    print('\n')

In [11]:
from question_from_headline import create_question_from_headline

def get_q_from_headline():
  headlines = [
      'Six streets hit hardest by anti-social behaviour in city',
      'Man wanted for fatal assault in coastal village spotted in North East shop'
  ]

  questioner_type = 'journalist'

  questions_list = []
  for headline in headlines:
    q = create_question_from_headline(headline, questioner_type, openai_key)
    questions_list.append(q)

  for i, item in enumerate(questions_list):
    print(f"{i+1}. {item}")


# Keyword Relate


In [12]:
def relate_keyword(keyword_input):
  url = "https://api.openai.com/v1/chat/completions"
  text = 'provide 10 thematically related words to the following text:' + keyword_input
  headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai_key}"
    }
  data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": text
            }
        ]
    }
  response = requests.post(url, headers=headers, json=data)
  return response.json()['choices'][0]['message']['content']


text = relate_keyword('police')
print(text)



1. Law enforcement  
2. Officer  
3. Crime  
4. Patrol  
5. Uniform  
6. Investigation  
7. Security  
8. Arrest  
9. Badge  
10. Force


In [18]:
# the inputs
button = widgets.Button(description="Run")
output = widgets.Output()

keyword = 'Policing' # @param {type:"string"}
demographic = '18-24' # @param ["18-24", '25-34', '35-44', '45-54',	'55-64', '65+']
region = 'North-East' # @param {type:"string"}
# date_input = '2024-04-01' # @param {type:"date"}

def run_all(b):
  with output:
    print(color.BOLD +  "BRIEF SUMMARY:" + color.END)
    get_overall_summary(keyword, region)
    print(color.BOLD +  "RELEVANT NEWS:" + color.END)
    get_newscatcher_outputs(keyword, region)
    print(color.BOLD +  "POLLING INFO:" + color.END)
    get_polling(demographic, region, keyword)
    print(color.BOLD +  "GENERAL QUESTIONS:" + color.END)
    get_questions_all(keyword, region)
    print(color.BOLD +  "QUESTIONS FROM HEADLINES:" + color.END)
    get_q_from_headline()

button.on_click(run_all)
display(button, output)

Button(description='Run', style=ButtonStyle())

Output()